In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
file_review3="C:/Users/Lu/PycharmProjects/Group_Project/orig_review_with_labeling_608598rows.txt"
review3=pd.read_csv(file_review3,sep="\t",header=None)
review3.columns=['user_id', 'prod_id', 'date', 'review', 'rating', 'label']
print(review3.head())
print("length of data:",len(review3))
print("check the null data:",review3.isnull().values.any())

   user_id  prod_id        date  \
0     5044        0  2014-11-16   
1     5045        0  2014-09-08   
2     5046        0  2013-10-06   
3     5047        0  2014-11-30   
4     5048        0  2014-08-28   

                                              review  rating  label  
0  Drinks were bad, the hot chocolate was watered...     1.0     -1  
1  This was the worst experience I've ever had a ...     1.0     -1  
2  This is located on the site of the old Spruce ...     3.0     -1  
3  I enjoyed coffee and breakfast twice at Toast ...     5.0     -1  
4  I love Toast! The food choices are fantastic -...     5.0     -1  
length of data: 608598
check the null data: False


In [10]:
data=review3[['user_id','review']]
print(data.head())

   user_id                                             review
0     5044  Drinks were bad, the hot chocolate was watered...
1     5045  This was the worst experience I've ever had a ...
2     5046  This is located on the site of the old Spruce ...
3     5047  I enjoyed coffee and breakfast twice at Toast ...
4     5048  I love Toast! The food choices are fantastic -...


### check total number of reviews posted by each user_id

In [31]:
review_total=data.groupby([data.user_id])['review'].count()
review_total=review_total.to_frame()
review_total.reset_index(inplace=True)
review_total=review_total.rename(columns={'user_id':'user_id', 'review':'total_num_reviews'})
print(review_total.head())
print('number of reviewers:', len(review_total))

   user_id  total_num_reviews
0     5044                  1
1     5045                  1
2     5046                  4
3     5047                  1
4     5048                  1
number of reviewers: 260277


In [32]:
data2=review3[['user_id','review','rating']]
print(data2.head())

   user_id                                             review  rating
0     5044  Drinks were bad, the hot chocolate was watered...     1.0
1     5045  This was the worst experience I've ever had a ...     1.0
2     5046  This is located on the site of the old Spruce ...     3.0
3     5047  I enjoyed coffee and breakfast twice at Toast ...     5.0
4     5048  I love Toast! The food choices are fantastic -...     5.0


In [33]:
positive_reviews=data2[data2['rating']>=4]
print(positive_reviews.head())

   user_id                                             review  rating
3     5047  I enjoyed coffee and breakfast twice at Toast ...     5.0
4     5048  I love Toast! The food choices are fantastic -...     5.0
5     5049  The egg on an English muffin (their take on eg...     5.0
6     5050  wonderful relaxed vibe and fantastic homemade ...     5.0
9     5053  First brunch experience here in Philly (actual...     4.0


In [34]:
negative_reviews=data2[data2['rating']<=2]
print(negative_reviews.head())

    user_id                                             review  rating
0      5044  Drinks were bad, the hot chocolate was watered...     1.0
1      5045  This was the worst experience I've ever had a ...     1.0
7      5051  Extremely slow kitchen. I went with an hour to...     1.0
8      5052  I really wanted to love toast. It's quaint and...     2.0
13     5057  .. I wanted to love this place... I stopped by...     2.0


### a) check number of positive reviews posted by each user_id

In [35]:
num_positive_reviews=positive_reviews.groupby([positive_reviews.user_id])['review'].count()
num_positive_reviews=num_positive_reviews.to_frame()
num_positive_reviews.reset_index(inplace=True)
num_positive_reviews=num_positive_reviews.rename(columns={'user_id':'user_id', 'review':'num_positive_reviews'})
print(num_positive_reviews.head(10))

   user_id  num_positive_reviews
0     5046                     1
1     5047                     1
2     5048                     1
3     5049                     1
4     5050                     1
5     5052                     2
6     5053                     8
7     5054                     5
8     5055                     1
9     5056                     8


### b) check number of negative reviews posted by each user_id

In [36]:
num_negative_reviews=negative_reviews.groupby([negative_reviews.user_id])['review'].count()
num_negative_reviews=num_negative_reviews.to_frame()
num_negative_reviews.reset_index(inplace=True)
num_negative_reviews=num_negative_reviews.rename(columns={'user_id':'user_id', 'review':'num_negative_reviews'})
print(num_negative_reviews.head(10))

   user_id  num_negative_reviews
0     5044                     1
1     5045                     1
2     5051                     1
3     5052                     1
4     5057                     1
5     5058                     3
6     5061                     5
7     5063                     1
8     5064                     1
9     5065                     2


In [40]:
df0=review_total.merge(num_positive_reviews, on='user_id',how='left')
print(df0.iloc[5:10,:])

   user_id  total_num_reviews  num_positive_reviews
5     5049                  1                   1.0
6     5050                  1                   1.0
7     5051                  1                   NaN
8     5052                  3                   2.0
9     5053                  8                   8.0


In [42]:
final_data=df0.merge(num_negative_reviews, on='user_id',how='left')
print(final_data.iloc[5:15,:])

    user_id  total_num_reviews  num_positive_reviews  num_negative_reviews
5      5049                  1                   1.0                   NaN
6      5050                  1                   1.0                   NaN
7      5051                  1                   NaN                   1.0
8      5052                  3                   2.0                   1.0
9      5053                  8                   8.0                   NaN
10     5054                  6                   5.0                   NaN
11     5055                  1                   1.0                   NaN
12     5056                 10                   8.0                   NaN
13     5057                  2                   1.0                   1.0
14     5058                  7                   2.0                   3.0


### replace NaN to 0

In [43]:
final_data['num_positive_reviews'] = final_data['num_positive_reviews'].fillna(0)
final_data['num_negative_reviews'] = final_data['num_negative_reviews'].fillna(0)
print(final_data.iloc[5:15,:])

    user_id  total_num_reviews  num_positive_reviews  num_negative_reviews
5      5049                  1                   1.0                   0.0
6      5050                  1                   1.0                   0.0
7      5051                  1                   0.0                   1.0
8      5052                  3                   2.0                   1.0
9      5053                  8                   8.0                   0.0
10     5054                  6                   5.0                   0.0
11     5055                  1                   1.0                   0.0
12     5056                 10                   8.0                   0.0
13     5057                  2                   1.0                   1.0
14     5058                  7                   2.0                   3.0


In [46]:
final_data['ratio_positive_reviews']=final_data['num_positive_reviews']/final_data['total_num_reviews']
final_data['ratio_negative_reviews']=final_data['num_negative_reviews']/final_data['total_num_reviews']
print(final_data.iloc[5:15,:])

    user_id  total_num_reviews  num_positive_reviews  num_negative_reviews  \
5      5049                  1                   1.0                   0.0   
6      5050                  1                   1.0                   0.0   
7      5051                  1                   0.0                   1.0   
8      5052                  3                   2.0                   1.0   
9      5053                  8                   8.0                   0.0   
10     5054                  6                   5.0                   0.0   
11     5055                  1                   1.0                   0.0   
12     5056                 10                   8.0                   0.0   
13     5057                  2                   1.0                   1.0   
14     5058                  7                   2.0                   3.0   

    ratio_positive_reviews  ratio_negative_reviews  
5                 1.000000                0.000000  
6                 1.000000         

### merge the data to original

In [51]:
original=review3[['user_id','prod_id','date','rating','label']].copy()
print(original)

        user_id  prod_id        date  rating  label
0          5044        0  2014-11-16     1.0     -1
1          5045        0  2014-09-08     1.0     -1
2          5046        0  2013-10-06     3.0     -1
3          5047        0  2014-11-30     5.0     -1
4          5048        0  2014-08-28     5.0     -1
...         ...      ...         ...     ...    ...
608593   119664     5039  2013-01-20     4.0      1
608594    56277     5039  2012-11-12     2.0      1
608595   265320     5039  2012-08-22     1.0      1
608596   161722     5039  2011-05-11     4.0      1
608597    78454     5039  2010-07-17     4.0      1

[608598 rows x 5 columns]


In [52]:
merged_data=original.merge(final_data, on='user_id',how='left')
print(merged_data.head())

   user_id  prod_id        date  rating  label  total_num_reviews  \
0     5044        0  2014-11-16     1.0     -1                  1   
1     5045        0  2014-09-08     1.0     -1                  1   
2     5046        0  2013-10-06     3.0     -1                  4   
3     5047        0  2014-11-30     5.0     -1                  1   
4     5048        0  2014-08-28     5.0     -1                  1   

   num_positive_reviews  num_negative_reviews  ratio_positive_reviews  \
0                   0.0                   1.0                    0.00   
1                   0.0                   1.0                    0.00   
2                   1.0                   0.0                    0.25   
3                   1.0                   0.0                    1.00   
4                   1.0                   0.0                    1.00   

   ratio_negative_reviews  
0                     1.0  
1                     1.0  
2                     0.0  
3                     0.0  
4     

In [55]:
print(len(merged_data))
print(merged_data['total_num_reviews'].isnull().values.any())
print(merged_data['ratio_positive_reviews'].isnull().values.any())
print(merged_data['ratio_negative_reviews'].isnull().values.any())

608598
False
False
False


In [57]:
from numpy import savetxt
savetxt("feature_ratio_positive_negative_reviews.txt",merged_data,fmt="%s",delimiter="\t",encoding="utf-8")